# Question
Chargement des fichiers pour les années 2021, 2022 

In [ ]:
import requests
import os

# URLs des fichiers CSV pour les années 2021 et 2022
urls = {
    'usagers-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094229/usagers-2022.csv",
    'vehicules-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094147/vehicules-2022.csv",
    'lieux-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094112/lieux-2022.csv",
    'caracteristiques-2022.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-093927/carcteristiques-2022.csv",
    'usagers-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2022/20231009-140337/usagers-2021.csv",
    'vehicules-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113925/vehicules-2021.csv",
    'lieux-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113901/lieux-2021.csv",
    'caracteristiques-2021.csv': "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2020/20221024-113743/carcteristiques-2021.csv"
}

# Création du répertoire 'data' s'il n'existe pas
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

# Téléchargement des fichiers
for filename, url in urls.items():
    response = requests.get(url)
    if response.status_code == 200:
        with open(os.path.join(data_dir, filename), 'wb') as f:
            f.write(response.content)
        print(f"Téléchargement réussi : {filename}")
    else:
        print(f"Erreur lors du téléchargement de {filename}")


# Question Supplémentaire
Netoyage du repertoire data après un eventuel telechargement des fichiers .csv

In [ ]:
import os

# Chemin du répertoire 'data'
data_dir = 'data'

# Vérifier si le répertoire existe
if os.path.exists(data_dir):
    # Itérer sur tous les fichiers du répertoire
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        # Vérifier si c'est un fichier (et non un sous-répertoire)
        if os.path.isfile(file_path):
            os.remove(file_path)  # Supprimer le fichier
            print(f"Fichier {filename} supprimé.")
else:
    print(f"Le répertoire {data_dir} n'existe pas.")


# Question
Création de dataframes correspondants au fichiers des années 2021, 2022 et
nettoyage des données

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType, LongType
from pyspark.sql.functions import lpad, concat, lit, to_date
from pyspark.ml.feature import StringIndexer

# Initialiser SparkSession
spark = SparkSession.builder.appName("Spark session....").getOrCreate()
#spark = SparkSession.builder.master("local[1]").appName("SparkByExamples.com").getOrCreate()

# Définir le schéma sans spécifier de types pour lat et long
schema_caracteristiques = StructType([
    StructField("Accident_Id", LongType(), True),
    StructField("jour", IntegerType(), True),
    StructField("mois", IntegerType(), True),
    StructField("an", IntegerType(), True),
    StructField("hrmn", StringType(), True),
    StructField("lum", IntegerType(), True),
    StructField("dep", IntegerType(), True),
    StructField("com", IntegerType(), True),
    StructField("agg", IntegerType(), True),
    StructField("int", IntegerType(), True),
    StructField("atm", IntegerType(), True),
    StructField("col", IntegerType(), True),
    StructField("adr", StringType(), True),
    StructField("lat", StringType(), True),  # Lire comme StringType
    StructField("long", StringType(), True)  # Lire comme StringType
])



schema_lieux = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("catr", IntegerType(), True),  
    StructField("voie", StringType(), True),
    StructField("V1", StringType(), True),
    StructField("V2", StringType(), True),
    StructField("circ", IntegerType(), True),
    StructField("nbv", IntegerType(), True),
    StructField("vosp", IntegerType(), True),
    StructField("prof", IntegerType(), True),
    StructField("pr", IntegerType(), True),
    StructField("pr1", IntegerType(), True),
    StructField("plan", IntegerType(), True),
    StructField("lartpc", IntegerType(), True),
    StructField("larrout", IntegerType(), True),
    StructField("surf", IntegerType(), True),
    StructField("infra", IntegerType(), True), 
    StructField("situ", IntegerType(), True),
    StructField("vma", IntegerType(), True)
])


schema_usagers = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("id_usager", StringType(), True),  
    StructField("id_vehicule", StringType(), True),  
    StructField("num_veh", StringType(), True),
    StructField("place", IntegerType(), True),
    StructField("catu", IntegerType(), True),
    StructField("grav", IntegerType(), True),
    StructField("sexe", IntegerType(), True),
    StructField("an_nais", IntegerType(), True),
    StructField("trajet", IntegerType(), True),
    StructField("secu1", IntegerType(), True),
    StructField("secu2", IntegerType(), True),
    StructField("secu3", IntegerType(), True),
    StructField("locp", IntegerType(), True),
    StructField("actp", IntegerType(), True), 
    StructField("etatp", IntegerType(), True)
])



schema_vehicules = StructType([
    StructField("Num_Acc", LongType(), True),
    StructField("id_vehicule", StringType(), True),
    StructField("num_veh", StringType(), True),
    StructField("senc", IntegerType(), True),
    StructField("catv", IntegerType(), True),
    StructField("obs", IntegerType(), True),
    StructField("obsm", IntegerType(), True),
    StructField("choc", IntegerType(), True),
    StructField("manv", IntegerType(), True),
    StructField("motor", IntegerType(), True),
    StructField("occutc", IntegerType(), True)
])





# Fonction pour charger et nettoyer les données
def charger_et_nettoyer(schemaDataFrame, fichierCsv):
    dataFrame = spark.read.csv(fichierCsv, header=True, sep=";", schema=schemaDataFrame)


    
    # Nettoyage de 'long' et 'lat'
    if "long" in dataFrame.columns:
        dataFrame = dataFrame.withColumn("long", regexp_replace("long", ",", ".").cast("double"))
    if "lat" in dataFrame.columns:
        dataFrame = dataFrame.withColumn("lat", regexp_replace("lat", ",", ".").cast("double"))
    
    categorial = []
    # Traitement pour schema_caracteristiques
    if schemaDataFrame == schema_caracteristiques:
        categorial = ["lum", "agg", "int", "atm", "col"]
        dataFrame = dataFrame.withColumn("jour", lpad(col("jour"), 2, "0"))
        dataFrame = dataFrame.withColumn("mois", lpad(col("mois"), 2, "0"))
        date_str = concat(lpad(col("jour"), 2, "0"), lit("/"), lpad(col("mois"), 2, "0"), lit("/"), col("an"))
        dataFrame = dataFrame.withColumn("date", to_date(date_str, "dd/MM/yyyy"))
    
    # Traitement pour schema_lieux
    elif schemaDataFrame == schema_lieux:
        categorial = ["catr", "circ", "vosp", "prof", "plan", "surf", "infra", "situ"]
    
    # Traitement pour schema_vehicules
    elif schemaDataFrame == schema_vehicules:
        categorial = ["senc", "catv", "obs", "obsm", "choc", "manv", "motor"]
    
    # Traitement pour schema_usagers
    elif schemaDataFrame == schema_usagers:
        categorial = ["catu", "grav", "sexe", "trajet", "secu1", "secu2", "secu3", "locp", "actp", "etatp"]
    
    # Application de StringIndexer avec option "keep"
    for column in categorial:
        label_stringIdx = StringIndexer(inputCol=column, outputCol=column+"_index", handleInvalid="keep")
        model = label_stringIdx.fit(dataFrame)
        dataFrame = model.transform(dataFrame)
    
    return dataFrame


def affichageDataFram(dataFrame, fichierCsv) : 
    print("#############################DataFrame du fichier .csv : ", fichierCsv, "#############################")
    dataFrame.show()


csvPathCararteristique2021 = "data/caracteristiques-2021.csv"
csvPathLieux2021 = "data/lieux-2021.csv"
csvPathUsagers2021 = "data/usagers-2021.csv"
csvPathVehicules2021 = "data/vehicules-2021.csv"

csvPathCararteristique2022 = "data/caracteristiques-2022.csv"
csvPathLieux2022 = "data/lieux-2022.csv"
csvPathUsagers2022 = "data/usagers-2022.csv"
csvPathVehicules2022 = "data/vehicules-2022.csv"


#Année 2021
df_caracteristiques_2021 = charger_et_nettoyer(schema_caracteristiques, csvPathCararteristique2021)
df_lieux_2021 = charger_et_nettoyer(schema_lieux, csvPathLieux2021)
df_usagers_2021 = charger_et_nettoyer(schema_usagers, csvPathUsagers2021)
df_vehicules_2021 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2021)

affichageDataFram(df_caracteristiques_2021, csvPathCararteristique2021)
affichageDataFram(df_lieux_2021, csvPathLieux2021)
affichageDataFram(df_usagers_2021, csvPathUsagers2021)
affichageDataFram(df_vehicules_2021, csvPathVehicules2021)

print("###########################################################################################################################################")


#Année 2022
df_caracteristiques_2022 = charger_et_nettoyer(schema_caracteristiques, csvPathCararteristique2022)
df_lieux_2022 = charger_et_nettoyer(schema_lieux, csvPathLieux2022)
df_usagers_2022 = charger_et_nettoyer(schema_usagers, csvPathUsagers2022)
df_vehicules_2022 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2022)

affichageDataFram(df_caracteristiques_2022, csvPathCararteristique2022)
affichageDataFram(df_lieux_2022, csvPathLieux2022)
affichageDataFram(df_usagers_2022, csvPathUsagers2022)
affichageDataFram(df_vehicules_2022, csvPathVehicules2022)

24/03/31 21:46:52 WARN Utils: Your hostname, myhost resolves to a loopback address: 127.0.1.1; using 192.168.1.23 instead (on interface wlp0s20f3)
24/03/31 21:46:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/31 21:46:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/31 21:46:54 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/31 21:46:54 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


#############################DataFrame du fichier .csv :  data/caracteristiques-2021.csv #############################
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
| Accident_Id|jour|mois|  an| hrmn|lum|dep|  com|agg|int|atm|col|                 adr|          lat|        long|      date|lum_index|agg_index|int_index|atm_index|col_index|
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|202100000001|  30|  11|2021|07:32|  2| 30|30319|  1|  1|  1|  1|              CD 981|    44.038958|    4.348022|2021-11-30|      3.0|      1.0|      0.0|      0.0|      3.0|
|202100000002|  25|  09|2021|14:20|  1| 51|51544|  1|  3|  1|  3|Aire de repos cro...|    49.242129|    4.554546|2021-09-25|      0.0|      1.0|      2.0|      0.0|      0.0|
|20210

24/03/31 21:47:15 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------------+----+--------------------+---+---+----+---+----+----+----+----+----+------+-------+----+-----+----+---+----------+----------+----------+----------+----------+----------+-----------+----------+
|     Num_Acc|catr|                voie| V1| V2|circ|nbv|vosp|prof|  pr| pr1|plan|lartpc|larrout|surf|infra|situ|vma|catr_index|circ_index|vosp_index|prof_index|plan_index|surf_index|infra_index|situ_index|
+------------+----+--------------------+---+---+----+---+----+----+----+----+----+------+-------+----+-----+----+---+----------+----------+----------+----------+----------+----------+-----------+----------+
|202100000001|   3|                 981| -1|N/A|   2|  2|   0|   1|NULL|NULL|   1|  NULL|   NULL|   1|    0|   1| 80|       1.0|       0.0|       0.0|       0.0|       0.0|       0.0|        0.0|       0.0|
|202100000002|   3|                  20|  0|  E|   2|  2|   0|   1|   0|  10|   1|  NULL|   NULL|   1|    0|   1| 80|       1.0|       0.0|       0.0|       0.0|       0.0|

# Question
Réunion des années 2021, 20222

In [2]:
csvPathCararteristique2021 = "data/caracteristiques-2021.csv"
csvPathLieux2021 = "data/lieux-2021.csv"
csvPathUsagers2021 = "data/usagers-2021.csv"
csvPathVehicules2021 = "data/vehicules-2021.csv"

csvPathCararteristique2022 = "data/caracteristiques-2022.csv"
csvPathLieux2022 = "data/lieux-2022.csv"
csvPathUsagers2022 = "data/usagers-2022.csv"
csvPathVehicules2022 = "data/vehicules-2022.csv"

#Année 2021
df_caracteristiques_2021 = charger_et_nettoyer(schema_caracteristiques, csvPathCararteristique2021)
df_lieux_2021 = charger_et_nettoyer(schema_lieux, csvPathLieux2021)
df_usagers_2021 = charger_et_nettoyer(schema_usagers, csvPathUsagers2021)
df_vehicules_2021 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2021)

#Année 2022
df_caracteristiques_2022 = charger_et_nettoyer(schema_caracteristiques, csvPathCararteristique2022)
df_lieux_2022 = charger_et_nettoyer(schema_lieux, csvPathLieux2022)
df_usagers_2022 = charger_et_nettoyer(schema_usagers, csvPathUsagers2022)
df_vehicules_2022 = charger_et_nettoyer(schema_vehicules, csvPathVehicules2022)



# Réaliser une union (réunion) des DataFrames de chaque catégorie pour les années 2021 et 2022
df_caracteristiques_union = df_caracteristiques_2021.union(df_caracteristiques_2022)
df_lieux_union = df_lieux_2021.union(df_lieux_2022)
df_usagers_union = df_usagers_2021.union(df_usagers_2022)
df_vehicules_union = df_vehicules_2021.union(df_vehicules_2022)

# Afficher les résultats pour vérifier
print("DataFrame Caractéristiques Union 2021 et 2022")
df_caracteristiques_union.show()

print("DataFrame Lieux Union 2021 et 2022")
df_lieux_union.show()

print("DataFrame Usagers Union 2021 et 2022")
df_usagers_union.show()

print("DataFrame Vehicules Union 2021 et 2022")
df_vehicules_union.show()

DataFrame Caractéristiques Union 2021 et 2022
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
| Accident_Id|jour|mois|  an| hrmn|lum|dep|  com|agg|int|atm|col|                 adr|          lat|        long|      date|lum_index|agg_index|int_index|atm_index|col_index|
+------------+----+----+----+-----+---+---+-----+---+---+---+---+--------------------+-------------+------------+----------+---------+---------+---------+---------+---------+
|202100000001|  30|  11|2021|07:32|  2| 30|30319|  1|  1|  1|  1|              CD 981|    44.038958|    4.348022|2021-11-30|      3.0|      1.0|      0.0|      0.0|      3.0|
|202100000002|  25|  09|2021|14:20|  1| 51|51544|  1|  3|  1|  3|Aire de repos cro...|    49.242129|    4.554546|2021-09-25|      0.0|      1.0|      2.0|      0.0|      0.0|
|202100000003|  15|  07|2021|07:55|  1| 85|85048|  2|  1|  7|  6|15 rue Françoi

# Question
Résumés numériques

In [ ]:
from pyspark.sql.functions import mean, stddev, skewness, kurtosis, col, median
import plotly.graph_objects as go

#Answer
def stats(df):
    numeric_cols = [c for c, t in df.dtypes if t.startswith('string') == False and t != "date"]
    stats_df = []
    for col_name in numeric_cols:
        mean_val = df.select(mean(col_name)).collect()[0][0]
        stddev_val = df.select(stddev(col_name)).collect()[0][0]
        skewness_val = df.select(skewness(col_name)).collect()[0][0]
        kurtosis_val = df.select(kurtosis(col_name)).collect()[0][0]
        quantiles = df.stat.approxQuantile(col_name, [0.25, 0.5, 0.75], 0.05)
        
        iqr = quantiles[2] - quantiles[0]  # Interquartile range
        stats_df.append((col_name, mean_val, quantiles[1], quantiles[0], quantiles[2], iqr, stddev_val, skewness_val, kurtosis_val))
        
        # Plotly Boxplot
        fig = go.Figure()
        fig.add_trace(go.Box(y=df.select(col_name).rdd.flatMap(lambda x: x).collect(), name=col_name))
        fig.update_layout(title=f"Boxplot of {col_name}")
        fig.show()
        
    stats_df = spark.createDataFrame(stats_df, ["Column", "Mean", "Median", "Q1", "Q3", "IQR", "StdDev", "Skewness", "Kurtosis"])
    stats_df.show()

#Données statistiques pour dataframe 2021
stats(df_caracteristiques_2021.na.drop())
stats(df_lieux_2021.na.drop())
stats(df_usagers_2021.na.drop())
stats(df_vehicules_2021.na.drop())


#Données statistiques pour dataframe 2022
stats(df_caracteristiques_2022.na.drop())
stats(df_lieux_2022.na.drop())
stats(df_usagers_2022.na.drop())
stats(df_vehicules_2022.na.drop())

"""
stats(df_caracteristiques_union.na.drop())
stats(df_lieux_union.na.drop())
stats(df_usagers_union.na.drop())
stats(df_vehicules_union.na.drop())"""

# Question
Visualisation
Construisez des objets graphiques propres à visualiser :
- Répartition des accidents sur la semaine (jours et heures)
- Répartition des accidents sur les mois de l’année

In [ ]:
#Answer 


# Question
Profil des usagers

In [ ]:
#Answer 


# Question
Accidents impliquant des cyclistes et/ou des piétons

In [ ]:
#Answer 


# Question
Usage des types composites

In [ ]:
#Answer 


# Question
Sauvegarde au format parquet

In [ ]:
#Answer 


# Question
Autres questions

In [ ]:
#Answer 
